# Household location choice model comparisons

Sam Maurer, June 2016

This notebook compares the MTC and UAL model specifications and estimation results, and estimates new tenue-sensitive choice models with specifications similar to MTC's.

In [1]:
%load_ext autoreload
%aimport baus
%autoreload 1

In [2]:
import orca
from baus import models
from urbansim.utils import misc
from urbansim.models.dcm import SegmentedMNLDiscreteChoiceModel

In [3]:
# To load the config files, urbansim.utils.misc expects to be running from the
# root of an urbansim instance
import os
os.chdir("/Users/smmaurer/Dropbox/Git-rMBP/ual/bayarea_urbansim")

In [4]:
import pprint
pp = pprint.PrettyPrinter()

In [5]:
def load_slcm(filename):
    # Load segmented LCM from yaml file
    return SegmentedMNLDiscreteChoiceModel.from_yaml(str_or_buffer=misc.config(filename))

In [6]:
def report_fit(slcm):
    # SegmentedMNLDiscreteChoiceModel() does not have its own report_fit() method
    for name, segment in slcm._group.models.items():
        print("LCM RESULTS FOR SEGMENT %s\n" % str(name))
        segment.report_fit()

### MTC model from Mar 2016

In [37]:
hlcm = load_slcm("hlcm.yaml")

In [45]:
d = hlcm.to_dict()
#pp.pprint(d)

In [38]:
report_fit(hlcm)

LCM RESULTS FOR SEGMENT 1

Null Log-liklihood: -23472.138
Log-liklihood at convergence: -22659.697
Log-liklihood Ratio: 0.035

+----------------------+-------------+------------+---------+
| Component            | Coefficient | Std. Error | T-Score |
+----------------------+-------------+------------+---------+
| ave_income_1500      |    -0.133   |   0.027    |  -4.880 |
| embarcadero          |    -0.028   |   0.001    | -28.288 |
| jobs_45              |    -0.035   |   0.021    |  -1.642 |
| np.log1p(unit_price) |    -0.668   |   0.023    | -29.108 |
| pacheights           |    -0.005   |   0.001    |  -6.797 |
| stanford             |    -0.011   |   0.001    | -18.066 |
+----------------------+-------------+------------+---------+
LCM RESULTS FOR SEGMENT 2

Null Log-liklihood: -23472.138
Log-liklihood at convergence: -22909.357
Log-liklihood Ratio: 0.024

+----------------------+-------------+------------+---------+
| Component            | Coefficient | Std. Error | T-Score |
+-

### UAL model from summer 2015 (owners)

In [40]:
ual_hlcm_owner_old = load_slcm("ual_hlcm_owner_old.yaml")

In [47]:
print ual_hlcm_owner_old.to_dict()['choosers_fit_filters']

['hownrent == 1']


In [41]:
report_fit(ual_hlcm_owner_old)

LCM RESULTS FOR SEGMENT 1

Null Log-liklihood: -69003.870
Log-liklihood at convergence: -66344.640
Log-liklihood Ratio: 0.039

+---------------------------------+-------------+------------+----------+
| Component                       | Coefficient | Std. Error | T-Score  |
+---------------------------------+-------------+------------+----------+
| autoOffPeakRetail               |    0.067    |   0.056    |  1.205   |
| autoPeakTotal                   |    0.014    |   0.052    |  0.265   |
| ave_income                      |    -0.722   |   0.031    | -23.394  |
| jobs                            |    -0.020   |   0.006    |  -3.393  |
| np.log1p(asian * pct_asian)     |    4.844    |   0.010    | 481.949  |
| np.log1p(ave_lot_size_per_unit) |    1.253    |   0.112    |  11.209  |
| np.log1p(black * pct_black)     |    6.500    |   0.011    | 593.568  |
| np.log1p(hisp * pct_hisp)       |    4.434    |   0.007    | 635.321  |
| np.log1p(persons * ave_hhsize)  |    3.000    |   0.021  

### UAL model from summer 2015 (renters)

In [42]:
ual_hlcm_renter_old = load_slcm("ual_hlcm_renter_old.yaml")

In [48]:
print ual_hlcm_renter_old.to_dict()['choosers_fit_filters']

['(hownrent == 2 & income > 0)']


In [43]:
report_fit(ual_hlcm_renter_old)

LCM RESULTS FOR SEGMENT 1

Null Log-liklihood: -68985.449
Log-liklihood at convergence: -63703.556
Log-liklihood Ratio: 0.077

+---------------------------------+-------------+------------+----------+
| Component                       | Coefficient | Std. Error | T-Score  |
+---------------------------------+-------------+------------+----------+
| autoOffPeakRetail               |    -0.437   |   0.059    |  -7.425  |
| autoPeakTotal                   |    0.463    |   0.054    |  8.634   |
| ave_income                      |    -1.454   |   0.032    | -46.069  |
| jobs                            |    0.022    |   0.008    |  2.685   |
| np.log1p(asian * pct_asian)     |    5.374    |   0.008    | 641.331  |
| np.log1p(ave_lot_size_per_unit) |    1.063    |   0.117    |  9.059   |
| np.log1p(black * pct_black)     |    5.386    |   0.009    | 619.337  |
| np.log1p(hisp * pct_hisp)       |    6.267    |   0.006    | 1001.648 |
| np.log1p(persons * ave_hhsize)  |    0.198    |   0.020  

### Estimating new models for owners vs renters

Presuming that MTC wants to keep their basic model specifications, here we estimate new models with the minimal alterations necessary to differentiate tenure.

Separately from this notebook, I made two copies of MTC's hlcm.yaml file with:
- chooser_fit_filters for tenure
- chooser_predict_filters for tenure (for simulation)
- alts_predict_filters for tenure (for simulation)
- residential price or rent in model expression

The estimation model step definitions use units rather than buildings, so that the price and rent fields will both be there.

In [7]:
%%capture
orca.run([
    "ual_initialize_residential_units",
    "ual_match_households_to_units",
    "neighborhood_vars",
    "regional_vars",
    "ual_rsh_simulate",
    "ual_rrh_simulate"
])

In [7]:
orca.run([
    "ual_hlcm_owner_estimate"
])

Running step 'ual_hlcm_owner_estimate'
LCM RESULTS FOR SEGMENT 1

Null Log-liklihood: -23448.666
Log-liklihood at convergence: -23272.553
Log-liklihood Ratio: 0.008

+----------------------------------+-------------+------------+---------+
| Component                        | Coefficient | Std. Error | T-Score |
+----------------------------------+-------------+------------+---------+
| jobs_45                          |    0.069    |   0.024    |  2.840  |
| ave_income_1500                  |    -0.190   |   0.040    |  -4.730 |
| np.log1p(unit_residential_price) |    -0.456   |   0.047    |  -9.796 |
| embarcadero                      |    0.007    |   0.001    |  5.732  |
| pacheights                       |    -0.003   |   0.001    |  -3.265 |
| stanford                         |    -0.002   |   0.001    |  -2.605 |
+----------------------------------+-------------+------------+---------+
LCM RESULTS FOR SEGMENT 2

Null Log-liklihood: -23444.754
Log-liklihood at convergence: -23177

In [8]:
orca.run([
    "ual_hlcm_renter_estimate"
])

Running step 'ual_hlcm_renter_estimate'
LCM RESULTS FOR SEGMENT 1

Null Log-liklihood: -23429.106
Log-liklihood at convergence: -22349.782
Log-liklihood Ratio: 0.046

+---------------------------------+-------------+------------+---------+
| Component                       | Coefficient | Std. Error | T-Score |
+---------------------------------+-------------+------------+---------+
| jobs_45                         |    -0.123   |   0.019    |  -6.618 |
| ave_income_1500                 |    -1.483   |   0.020    | -75.502 |
| np.log1p(unit_residential_rent) |    0.974    |   0.078    |  12.552 |
| embarcadero                     |    -0.009   |   0.001    |  -9.700 |
| pacheights                      |    -0.004   |   0.001    |  -6.347 |
| stanford                        |    -0.005   |   0.001    |  -7.298 |
+---------------------------------+-------------+------------+---------+
LCM RESULTS FOR SEGMENT 2

Null Log-liklihood: -23413.458
Log-liklihood at convergence: -22741.786
Log-